In [7]:
import pandas as pd
import numpy as np

patient_count = 200;
interval = 3
eval_years = 4

np.random.seed(21)

# Simulating Evaluation upon entry into the database
baseline = pd.DataFrame({
    'patient_id': np.arange(0, patient_count),
    'pain_b': np.random.randint(0, 10, patient_count),
    'urgency_b': np.random.randint(0, 10, patient_count),
    'frequency_b': np.random.randint(0, 10, patient_count),
})
    

eval_columns = ['patient_id', 'pain', 'urgency', 'frequency', 'time_since_entry', 'time_treated', 'treated']
evaluations = pd.DataFrame(columns=eval_columns)
for p_id in range(patient_count):
    treatment_time = np.random.choice(list(np.arange(3, eval_years * 12 + 1, 3)) + [None])
    
    for month in range(3, eval_years * 12 + 1, 3):
        pain = np.random.randint(0, 10, 1)
        urgency = np.random.randint(0, 10, 1)
        frequency = np.random.randint(0, 10, 1)
        time_since_entry = month

        if treatment_time == None or month < treatment_time:
            time_treated = None
            treated = False
        else:
            time_treated = treatment_time
            treated = True

        evaluations = pd.concat([evaluations, pd.DataFrame({
            'patient_id': p_id,
            'pain': pain,
            'urgency': urgency,
            'frequency': frequency,
            'time_since_entry': month,
            'time_treated': time_treated,
            'treated': treated
        })])

baseline.to_csv('baseline_dataset.csv', index=False)
evaluations.to_csv('evaluations_dataset.csv', index=False)